In [17]:
import sys
import pandas as pd
import re
 
sys.path.append('../')

from functions import *

df = pd.read_csv("../../data/simulated.csv")

def comma(s):
    add_comma = lambda match: match.group(0).replace(" ", ", ")
    
    s = s.replace("\n", "")

    s = re.sub(r"([0-9\.]+\s)", add_comma, s)
    
    return eval(s)

df.residual = df.residual.apply(comma)

ERROR! Session/line number was not unique in database. History logging moved to new session 1109


In [2]:
include = df

bids = list(include.residual)

logged_feedback = include.sellerfeedbackscore
logged_feedback = transform_covariates(logged_feedback, 100)
include.sellerfeedbackscore = logged_feedback

covariates = np.array(include[["ispolice", "sellerfeedbackscore"]])
covariates = list([list(cov) for cov in covariates])
incremented = list(include.increment_residual)

In [3]:
median_upper, median_lower = estimate_median(bids, covariates, incremented, (-4,6))
dump(median_lower, median_upper, "CMF_s_main")

calculating values for covariate: [1.0, 5.144745708587448] (1/196)
total time elapsed: 0.0001840880000001377s
calculating values for covariate: [1.0, 9.04708671623842] (2/196)
total time elapsed: 71.948657631s
calculating values for covariate: [1.0, 4.579683341430985] (3/196)
total time elapsed: 142.324302798s
calculating values for covariate: [0.0, 4.142109650649297] (4/196)
total time elapsed: 211.707091795s
calculating values for covariate: [0.0, 7.983577673935434] (5/196)
total time elapsed: 296.552607512s
calculating values for covariate: [0.0, 6.67700734881775] (6/196)
total time elapsed: 405.819800962s
calculating values for covariate: [1.0, 6.189262940201957] (7/196)
total time elapsed: 520.907876506s
calculating values for covariate: [1.0, 7.526810981923995] (8/196)
total time elapsed: 620.010492355s
calculating values for covariate: [1.0, 9.487024917706549] (9/196)
total time elapsed: 713.123484541s
calculating values for covariate: [0.0, 10.719881006567508] (10/196)
total ti

calculating values for covariate: [0.0, 5.024759016353102] (80/196)
total time elapsed: 8327.964582577s
calculating values for covariate: [1.0, 2.1972245773362205] (81/196)
total time elapsed: 8436.960959572s
calculating values for covariate: [1.0, 0.41415110829800006] (82/196)
total time elapsed: 8526.955840673s
calculating values for covariate: [1.0, 6.7573622528918795] (83/196)
total time elapsed: 8619.566255031s
calculating values for covariate: [0.0, 6.98263933006936] (84/196)
total time elapsed: 8710.963595228s
calculating values for covariate: [1.0, 4.629835190084681] (85/196)
total time elapsed: 8814.69077141s
calculating values for covariate: [1.0, 6.509948407483026] (86/196)
total time elapsed: 8904.203107792s
calculating values for covariate: [1.0, 5.605183686550837] (87/196)
total time elapsed: 8998.537070717s
calculating values for covariate: [1.0, 7.058687446432212] (88/196)
total time elapsed: 9091.189475294s
calculating values for covariate: [0.0, 7.351269927280566] (89

calculating values for covariate: [1.0, 8.155362120328135] (158/196)
total time elapsed: 13609.991553428s
calculating values for covariate: [1.0, 5.519355154393757] (159/196)
total time elapsed: 13668.214859025s
calculating values for covariate: [0.0, 5.315525645153751] (160/196)
total time elapsed: 13724.6168324s
calculating values for covariate: [0.0, 5.244259141251391] (161/196)
total time elapsed: 13788.220608464s
calculating values for covariate: [1.0, 3.50616119555079] (162/196)
total time elapsed: 13852.844866831s
calculating values for covariate: [1.0, 7.145402053603267] (163/196)
total time elapsed: 13908.753763949s
calculating values for covariate: [1.0, 8.40092434936116] (164/196)
total time elapsed: 13965.021338817s
calculating values for covariate: [0.0, 5.274656687830727] (165/196)
total time elapsed: 14023.342829232s
calculating values for covariate: [0.0, 5.907577720953639] (166/196)
total time elapsed: 14086.779175477s
calculating values for covariate: [1.0, 4.34132378

In [8]:
percentile = lambda p: np.percentile(np.array(df.sellerfeedbackscore), p)

cef1 = lambda c, cov: c[0]*(1-cov[0])+c[1]*cov[0]+c[2]*cov[1]*(1-cov[0])+c[3]*cov[1]*cov[0]
n_cov1 = 4

def cef2(c, cov):
    _20_40 = cov[0] > percentile(20) and cov[0] <= percentile(40)
    _40_60 = cov[0] > percentile(40) and cov[0] <= percentile(60)
    _60_80 = cov[0] > percentile(60) and cov[0] <= percentile(80)
    _80_100 = cov[0] > percentile(80) and cov[0] <= percentile(100)
    
    return c[0]*cov[1] + c[1]*(1-cov[1]) + c[2]*cov[1]*_20_40 + c[3]*cov[1]*_40_60 + c[4]*cov[1]*_60_80 \
        + c[5]*cov[1]*_80_100 + c[6]*(1-cov[1])*_20_40 + c[7]*(1-cov[1])*_40_60 + c[8]*(1-cov[1])*_60_80 \
        + c[9]*(1-cov[1])*_80_100
n_cov2 = 10

def cef3(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0]
n_cov3 = 6

def cef4(c,cov):
    feedback = np.exp(cov[1])-1
    return c[0]*(1-cov[0]) + c[1]*cov[0] + c[2]*feedback*(1-cov[0]) + c[3]*feedback*cov[0] + \
        c[4]*feedback**2*(1-cov[0]) + c[5]*feedback**2*cov[0] + c[6]*feedback**3*(1-cov[0]) + \
        c[7]*feedback**3*cov[0]
n_cov3 = 8

In [ ]:
def loss_function1(c):
    cef = lambda cov: cef1(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point1, interval1 = get_estimates(loss_function1, n_cov1)

print(point1)
print(interval1)

In [11]:
def loss_function2(c):
    cef = lambda cov: cef2(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point2, interval2 = get_estimates(loss_function2, n_cov2)

print(point2)

print(interval2)

In [ ]:
def loss_function3(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point3, interval3 = get_estimates(loss_function3, n_cov3)

print(point3)
print(interval3)

In [ ]:
def loss_function4(c):
    cef = lambda cov: cef3(c, cov)
    return get_loss_function(covariates, median_upper, median_lower, cef)

point4, interval4 = get_estimates(loss_function4, n_cov4)

print(point4)
print(interval4)

In [ ]:
results = bootstrap(bids, covariates, incremented, [n_cov1, n_cov2, n_cov3, n_cov4], [cef1, cef2, cef3, cef4], 30, "s_main")